# Convergence and gradients of the Reinforce vs Gumbel-Softmax algorithm

Let's consider a simple optimization problem with the following loss:
$$\mathcal{L} = \mathbb{E}_{q(y)} \sum_d (y_k - t_k)^2$$
where $t$ is a $K$-dimensional target variable. We search for $q$ s.t. the objective is minimized.

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

In [2]:
tf.__version__, tfp.__version__

('2.2.0', '0.9.0')

In [3]:
# Discretization via straight-through:

def st(y):
    K = y.shape[-1]
    y_hard = tf.cast(tf.round(y), y.dtype) if K==1 else tf.cast(tf.one_hot(tf.argmax(y,-1), K), y.dtype)  
    y = tf.stop_gradient(y_hard - y) + y  
    return y

### Problem specification

In [4]:
# We can run the experiment for a 1D Categorical variable or for a Bernoulli variable:

## Categorical
target = tf.constant([0.2, 0.7, 0.1])
logits = tf.Variable([1.0]*len(target))
RDist = tfp.distributions.OneHotCategorical
Dist = tfp.distributions.RelaxedOneHotCategorical

## Bernoulli
# logits = tf.Variable([0.5])
# target = tf.constant([0.9])
# RDist = tfp.distributions.Bernoulli 
# Dist = tfp.distributions.RelaxedBernoulli

## REINFORCE

The Reinforce algorithm converges to zero loss and allocates all the probability mass at the most likely value.

In [5]:
eta = 0.01
for i in range(1000):    
    
    with tf.GradientTape() as tape:
        d = RDist(logits=logits)
        y = d.sample(1000)
        loss1 = tf.reduce_sum( (tf.cast(y, target.dtype)-target)**2, -1) * d.log_prob(y) 
        loss = tf.reduce_mean(loss1)
    
    grad = tape.gradient(loss, logits)    
    if i%100==0:
        print(f"loss={loss:.4f} grad={grad} "
          f"distribution={tf.reduce_mean( tf.cast(RDist(logits=logits).sample(10000), tf.float32), 0)}")
    
    logits.assign(logits-eta*grad)

loss=-0.9576 grad=[ 0.06742681 -0.24321333  0.17578736] distribution=[0.3292 0.3266 0.3442]
loss=-0.9616 grad=[ 0.13015397 -0.27806032  0.1479079 ] distribution=[0.2943 0.4202 0.2855]
loss=-0.9308 grad=[ 0.13325922 -0.28969526  0.15643695] distribution=[0.2534 0.5238 0.2228]
loss=-0.8066 grad=[ 0.11789955 -0.25187093  0.13397285] distribution=[0.2043 0.6173 0.1784]
loss=-0.7330 grad=[ 0.11393809 -0.2281118   0.11417679] distribution=[0.159  0.7075 0.1335]
loss=-0.6770 grad=[ 0.08494692 -0.20564716  0.12069937] distribution=[0.1343 0.757  0.1087]
loss=-0.5478 grad=[ 0.0703509  -0.15772922  0.08737966] distribution=[0.1008 0.8133 0.0859]
loss=-0.5235 grad=[ 0.06692715 -0.14770858  0.08078241] distribution=[0.0829 0.844  0.0731]
loss=-0.4632 grad=[ 0.0572156  -0.12591498  0.0686992 ] distribution=[0.0735 0.8661 0.0604]
loss=-0.4638 grad=[ 0.05256819 -0.12409036  0.0715222 ] distribution=[0.0588 0.8922 0.049 ]


## Gumbel softmax

The Gumbel-Softmax gradient is biased and therefore, the algorithm never achieves zero loss.

In [6]:
eta = 0.01
for i in range(1000):    
    
    with tf.GradientTape() as tape:
        p = Dist(logits=logits, temperature=0.5)
        y = st(p.sample(1000))
        loss1 = tf.reduce_sum( (y-target)**2, -1)    
        loss = tf.reduce_mean(loss1)

    grad = tape.gradient(loss, logits)    
    if i%100==0:
        print(f"loss={loss:.4f} grad={grad} "
          f"distribution={tf.reduce_mean( st(Dist(logits=logits, temperature=0.001).sample(10000)), 0)}")
    
    logits.assign(logits-eta*grad)

loss=0.2542 grad=[ 0.00612169 -0.02031396  0.01419225] distribution=[0.0544 0.9049 0.0407]
loss=0.2468 grad=[-0.00392383 -0.00294256  0.00686639] distribution=[0.0522 0.9051 0.0427]
loss=0.2436 grad=[-0.00976162 -0.00222388  0.01198549] distribution=[0.0504 0.9064 0.0432]
loss=0.2398 grad=[-0.00178365 -0.0076066   0.00939023] distribution=[0.0553 0.9034 0.0413]
loss=0.2362 grad=[-0.01069808  0.00196452  0.00873357] distribution=[0.0511 0.9081 0.0408]
loss=0.2488 grad=[ 0.00085526 -0.01531564  0.01446037] distribution=[0.051  0.9106 0.0384]
loss=0.2432 grad=[-0.00083958 -0.01359567  0.01443525] distribution=[0.0511 0.9124 0.0365]
loss=0.2292 grad=[-0.00717923  0.01286277 -0.00568354] distribution=[0.0532 0.9056 0.0412]
loss=0.2322 grad=[-0.00488815  0.00246094  0.0024272 ] distribution=[0.0512 0.9124 0.0364]
loss=0.2210 grad=[-0.01167171  0.00980752  0.00186419] distribution=[0.0495 0.9154 0.0351]


## MDNF

In [7]:
import sys
sys.path.append("../mdnf/")

from flows_factorized_mixture import FactorizedDiscreteFlowsMixture

In [8]:
eta = 0.01
p = FactorizedDiscreteFlowsMixture(N=1, K=len(target), B=10, temperature=.5)
logits = p.logits

In [9]:
opt = tf.keras.optimizers.Adam(learning_rate=eta)

In [10]:
for i in range(1000):    
    #p.temperature = 1000./(i+1)
    with tf.GradientTape() as tape:
        y = st(p.sample(1000)[:,0,:])
        loss1 = tf.reduce_sum( (y-target)**2, -1)    
        loss = tf.reduce_mean(loss1)

    grad = tape.gradient(loss, logits)    
    if i%100==0:
        print(f"loss={loss:.4f} "
              f"distribution={tf.reduce_mean(y, 0)}"
              #f"grad={grad} "
              #f"distribution={tf.reduce_mean( st(Dist(logits=logits, temperature=0.001).sample(10000)), 0)}"
             )
    
    #opt.apply_gradients(zip([grad], [logits]))
    logits.assign(logits-eta*grad)

loss=1.0200 distribution=[0.4 0.2 0.4]
loss=0.8800 distribution=[0.5 0.3 0.2]
loss=0.7800 distribution=[0.4 0.4 0.2]
loss=0.7800 distribution=[0.4 0.4 0.2]
loss=0.6800 distribution=[0.3 0.5 0.2]
loss=0.6800 distribution=[0.3 0.5 0.2]
loss=0.6800 distribution=[0.3 0.5 0.2]
loss=0.5600 distribution=[0.3 0.6 0.1]
loss=0.5600 distribution=[0.3 0.6 0.1]
loss=0.5600 distribution=[0.3 0.6 0.1]
